# **Ingestion del archivo "model.csv"**

##### **1.- Leer archivo csv**

In [1]:
#p_file_date = "2025-07-07"
#p_environment = "Development"

StatementMeta(, a92cc245-4d97-4fdb-adad-93e0cba79df2, 3, Finished, Available, Finished)

In [2]:
%run configuration

StatementMeta(, a92cc245-4d97-4fdb-adad-93e0cba79df2, 4, Finished, Available, Finished)

In [3]:
%run common_functions

StatementMeta(, a92cc245-4d97-4fdb-adad-93e0cba79df2, 5, Finished, Available, Finished)

In [4]:
#Importaciones de los tipos de datos para definir el esquema
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType

StatementMeta(, a92cc245-4d97-4fdb-adad-93e0cba79df2, 6, Finished, Available, Finished)

In [5]:
# Esta es la estructura del df, esto se define para que inferSchema no haga un proceso extra 
# sirve para grandes volumenes de datos, asi se demoran menos en cargar
model_schema = StructType ( fields = [
    StructField("ModelID", IntegerType(), False),
    StructField("CategoryID", IntegerType(), True),
    StructField("Model", StringType(), True),
    StructField("Codename", StringType(), True),
    StructField("GeneralExtras", StringType(), True)
])

StatementMeta(, a92cc245-4d97-4fdb-adad-93e0cba79df2, 7, Finished, Available, Finished)

In [6]:
# Ruta del archivo CSV en Azure Blob Storage o ADLS
abfs_path = f"{bronze_folder_path}/{p_file_date}/model.csv"

# Leer el archivo CSV desde ABFS
# El delimitador en este caso son de ; hay que agregar la option delimiter
df = spark.read.option("header", True) \
                .schema(model_schema) \
                .option("delimiter", ";") \
                .csv(abfs_path)
            

StatementMeta(, a92cc245-4d97-4fdb-adad-93e0cba79df2, 8, Finished, Available, Finished)

##### **2.- Seleccionar columnas requeridas, o eliminar generalExtras**

In [7]:
df2 = df.drop("GeneralExtras")

StatementMeta(, a92cc245-4d97-4fdb-adad-93e0cba79df2, 9, Finished, Available, Finished)

##### **3.- Cambiar nombres de columnas segun lo requerido**

In [8]:
new_cols = ["model_id", "category_id", "model", "code_name"]

df_renamed = df2.toDF(*new_cols)

StatementMeta(, a92cc245-4d97-4fdb-adad-93e0cba79df2, 10, Finished, Available, Finished)

##### **4.- Agregar columnas ingestion date y environment**

In [9]:
from pyspark.sql.functions import lit

df4 = add_ingestion_date(df_renamed) \
        .withColumn("environment", lit(p_environment)) \
        .withColumn("file_date", lit(p_file_date))

StatementMeta(, a92cc245-4d97-4fdb-adad-93e0cba79df2, 11, Finished, Available, Finished)

In [10]:
%%sql
DROP TABLE if EXISTS model

StatementMeta(, a92cc245-4d97-4fdb-adad-93e0cba79df2, 12, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

##### **5.- Pasar los archivos a delta a lh_silver**

In [11]:
df4.write.format("delta").mode("overwrite").saveAsTable("model")

StatementMeta(, a92cc245-4d97-4fdb-adad-93e0cba79df2, 13, Finished, Available, Finished)

In [12]:
%%sql
SELECT * from model
limit(10)

StatementMeta(, a92cc245-4d97-4fdb-adad-93e0cba79df2, 14, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 7 fields>